In [1]:
import os, sys
import ROOT
sys.path.append("../Utils/")
sys.path.append("../Config/")
import xconfig
import xutils
from fillReport import *
import math

fillReport = FillReport('../Config/FillReport_1446656923991.xls')
#
cfg = xconfig.Config('../Config/sourceCalibration.ini')
cfgData = cfg.get_option('Detectors')
detInd = {}
detNo = {}
detCf = {}
detCfErr = {}
for key in cfgData.keys():
    tmp = cfgData[key].split()
    detInd[key] = int(tmp[0])
    detNo[key] = int(tmp[1])
    detCf[key] = float(tmp[2])
    detCfErr[key] = float(tmp[3])
del cfg

In [9]:
inputPattern = '/home/data/RadMonData/RadmonFills/radmon__XXX__.root'
refIndx = 5   # PFIB used to find beginning of the collisions
rateCut = 50.  # If rate higher than this value, collisions started 
minPeakInstLumi = 10.  # Only fills with larger lumi are taken
deltaT = 100.  #  Warm-up cut  
minBfield = 3.6

#fills = [ 4528, 4530]
fills = [ 3960, 3962, 3965, 3971, 3974,3976, 3981, 3983, 3986, 3988, 3992, 3996, 
        4001, 4006, 4008, 4019, 4020, 4201, 4205, 4207, 4208, 4210, 4211, 4212, 
         4214, 4219, 4220, 4224, 4225, 4231, 4243, 4246, 4249, 4254, 4256, 4257, 
         4266, 4268, 4269, 4322, 4323, 4332, 4337, 4341, 4342, 4349, 4356, 4360, 
         4363, 4364, 4376, 4381, 4384, 4386, 4391, 4393, 4397, 4398, 4402, 4410, 
         4418, 4420, 4423, 4426, 4428, 4432, 4434, 4435, 4437,  4466, 4467,4476, 4477, 4479, 4485, 
         4495, 4496, 4499, 4505, 4509, 4510, 4511, 4513, 4518, 4519, 4522, 4525, 
         4528, 4530, 4532, 4536, 4538, 4540, 4545, 4555, 4557, 4562, 4565, ]
#         4440, 4444, 4448, 4449, 4452, 4455, 4462, 4463, 4464,]
#fills = [4440, 4444, 4448, 4449, 4452, 4455, 4462, 4463, 4464,]

# Ratio vs fill

In [10]:
# Ratio vs fill
graphs = {}
graphs["pnit/pnib"] = ROOT.TGraphErrors()
graphs["pfit/pfib"] = ROOT.TGraphErrors()
graphs["pfit/pnit"] = ROOT.TGraphErrors()
graphs["pfib/pnib"] = ROOT.TGraphErrors()

graphs["mnit/mnib"] = ROOT.TGraphErrors()
graphs["mfit/mfib"] = ROOT.TGraphErrors()
graphs["mfit/mnit"] = ROOT.TGraphErrors()
graphs["mfib/mnib"] = ROOT.TGraphErrors()

graphsLumi = {}
graphsLumi["pnit"] = ROOT.TGraph()
graphsLumi["pfit"] = ROOT.TGraph()
graphsLumi["pfit"] = ROOT.TGraph()
graphsLumi["pfib"] = ROOT.TGraph()
graphsLumi["mnit"] = ROOT.TGraph()
graphsLumi["mfit"] = ROOT.TGraph()
graphsLumi["mfit"] = ROOT.TGraph()
graphsLumi["mfib"] = ROOT.TGraph()

for key in graphs.keys():
    graphs[key].SetTitle(key.upper() +  " vs Fill number")
    graphs[key].SetMarkerStyle(8)
    
for key in graphsLumi.keys():
    graphsLumi[key].SetTitle(key.upper() +  "/deliveredLumi vs Fill number")
    graphsLumi[key].SetMarkerStyle(8)

print "Starting..."
for fill in fills:
    
    # Fill cuts
    if fillReport.getFillPeakInstLumi(fill) < minPeakInstLumi:
        #print "Low FillPeakInstLumi for fill", fill
        continue
    if fillReport.getFillField(fill) < minBfield:
        #print "Low magnetic field for fill", fill
        continue       
    if fillReport.getFillDuration(fill) < deltaT:
        #print "Short fill", fill
        continue
    
    inputFile = inputPattern.replace('__XXX__', str(fill))
    try:
        fin = ROOT.TFile(inputFile,'READ')
        #print "Processing fill", fillpeakInstLumi
    except IOError:
        print "Cannot open file", inputFile
        continue
    t = fin.Get("t")      
    
    # Sums
    sums = {"pnit":0.,
            "pnib":0., 
            "pfit":0.,
            "pfib":0.,
            "mnit":0.,
            "mnib":0., 
            "mfit":0.,
            "mfib":0.
            }
    
    tsColl = -10000
    for i in range(0, t.GetEntries()):
        nb = t.GetEntry(i)
        if nb < 0:
            continue
        
        #Start of collisions 
        if tsColl < 0:
            if t.rates[refIndx] > rateCut:
                tsColl = t.tstamp  
        if tsColl < 0 or t.tstamp < tsColl + deltaT:
            continue
            
        for key in sums.keys():
            sums[key] += t.rates[detInd[key]]

#==== Fill done

    for key in graphs.keys():
        (nom, denom) = key.split('/')
        if sums[denom] > 0.:
            y = sums[nom]/sums[denom]
            ddenom = 1/sums[denom]
        else:    
            y = 0.
            ddenom = 0.
        if sums[nom] > 0.:
            dnom = 1/sums[nom]
        else:
            dnom = 0.   
            

        dy = y*math.sqrt(dnom + ddenom)    
        
        n = graphs[key].GetN()
        graphs[key].SetPoint(n, float(fill), y)
        graphs[key].SetPointError(n, 0, dy)
        
    fillDeliveredLumi = fillReport.getFillDeliveredLumi(fill)
    for key in graphsLumi.keys():
        n = graphsLumi[key].GetN()
        graphsLumi[key].SetPoint(n, float(fill), sums[key]/fillDeliveredLumi)
        
print "==============Done==============="  

Starting...
==============Done===============


#Plot  ratios vs fill No

In [11]:
cp = ROOT.TCanvas("cp", "+Z", 800, 800)
cp.Divide(2,2)
cp.cd(1)
graphs["pnit/pnib"].Draw("AP")
graphs["pnit/pnib"].GetYaxis().SetRangeUser(0.5, .8)
cp.cd(2)
graphs["pfit/pfib"].Draw("AP")
graphs["pfit/pfib"].GetYaxis().SetRangeUser(0.5, .8)
cp.cd(3)
graphs["pfit/pnit"].Draw("AP")
graphs["pfit/pnit"].GetYaxis().SetRangeUser(1., 1.3)
cp.cd(4)
graphs["pfib/pnib"].Draw("AP")
graphs["pfib/pnib"].GetYaxis().SetRangeUser(1., 1.3)
cp.Update()
#
cm = ROOT.TCanvas("cm", "-Z", 800, 800)
cm.Divide(2,2)
cm.cd(1)
graphs["mnit/mnib"].Draw("AP")
graphs["mnit/mnib"].GetYaxis().SetRangeUser(0.7, 1.)
cm.cd(2)
graphs["mfit/mfib"].Draw("AP")
graphs["mfit/mfib"].GetYaxis().SetRangeUser(1.1, 1.5)
cm.cd(3)
graphs["mfit/mnit"].Draw("AP")
graphs["mfit/mnit"].GetYaxis().SetRangeUser(1.4, 2.)
cm.cd(4)
graphs["mfib/mnib"].Draw("AP")
graphs["mfib/mnib"].GetYaxis().SetRangeUser(0.7, 1.1)
cm.Update()   

In [7]:
c1 = ROOT.TCanvas()
graphs["pnit/pnib"].Draw("AP")
graphs["pnit/pnib"].GetYaxis().SetRangeUser(0.3, 1.)
graphs["pnit/pnib"].GetXaxis().SetTitle("Fill number")
#graphs["pnit/pnib"].Draw("AP")
c1.Update()

# Draw integrated rate vs delivered lumi

In [4]:
cpl = ROOT.TCanvas("cpl", "+Z", 800, 800)
cpl.Divide(2,2)
cpl.cd(1)
graphsLumi["pnit"].Draw("AP")
#graphsLumi["pnit/pnib"].GetYaxis().SetRangeUser(0.5, .8)
cpl.cd(2)
graphsLumi["pfit"].Draw("AP")
cpl.cd(3)
graphsLumi["pfit"].Draw("AP")
cpl.cd(4)
graphsLumi["pfib"].Draw("AP")
cpl.Update()
#
cml = ROOT.TCanvas("cml", "-Z", 800, 800)
cml.Divide(2,2)
cml.cd(1)
graphsLumi["mnit"].Draw("AP")
#graphsLumi["mnit/pnib"].GetYaxis().SetRangeUser(0.5, .8)
cml.cd(2)
graphsLumi["mfit"].Draw("AP")
cml.cd(3)
graphsLumi["mfit"].Draw("AP")
cml.cd(4)
graphsLumi["mfib"].Draw("AP")
cml.Update()


#Stat distributions

In [58]:
# 
histos = {}
nbins = 100
#no calibration factors
histos["pnit/pnib"] = ROOT.TH1D("pnit_pnib", "PNIT/PNIB", nbins, 0., -1.)
histos["pfit/pfib"] = ROOT.TH1D("pfit_pfib", "PFIT/PFIB", nbins, 0., -1.)
histos["pfit/pnit"] = ROOT.TH1D("pfit_pnit", "PFIT/PNIT", nbins, 0., -1.)
histos["pfib/pnib"] = ROOT.TH1D("pfib_pnib", "PFIB/PNIB", nbins, 0., -1.)
histos["mnit/mnib"] = ROOT.TH1D("mnit_mnib", "MNIT/MNIB", nbins, 0., -1.)
histos["mfit/mfib"] = ROOT.TH1D("mfit_mfib", "MFIT/MFIB", nbins, 0., -1.)
histos["mfit/mnit"] = ROOT.TH1D("mfit_mnit", "MFIT/MNIT", nbins, 0., -1.)
histos["mfib/mnib"] = ROOT.TH1D("mfib_mnib", "MFIB/MNIB", nbins, 0., -1.)

histos["pnit/pnib"].SetLineColor(ROOT.kRed)
histos["pfit/pfib"].SetLineColor(ROOT.kRed)
histos["pfit/pnit"].SetLineColor(ROOT.kRed)
histos["pfib/pnib"].SetLineColor(ROOT.kRed)
histos["mnit/mnib"].SetLineColor(ROOT.kRed)
histos["mfit/mfib"].SetLineColor(ROOT.kRed)
histos["mfit/mnit"].SetLineColor(ROOT.kRed)
histos["mfib/mnib"].SetLineColor(ROOT.kRed)

chistos = {}
#With calibration factors
chistos["pnit/pnib"] = ROOT.TH1D("c_pnit_pnib", "PNIT/PNIB", nbins, 0., -1.)
chistos["pfit/pfib"] = ROOT.TH1D("c_pfit_pfib", "PFIT/PFIB", nbins, 0., -1.)
chistos["pfit/pnit"] = ROOT.TH1D("c_pfit_pnit", "PFIT/PNIT", nbins, 0., -1.)
chistos["pfib/pnib"] = ROOT.TH1D("c_pfib_pnib", "PFIB/PNIB", nbins, 0., -1.)
chistos["mnit/mnib"] = ROOT.TH1D("c_mnit_mnib", "MNIT/MNIB", nbins, 0., -1.)
chistos["mfit/mfib"] = ROOT.TH1D("c_mfit_mfib", "MFIT/MFIB", nbins, 0., -1.)
chistos["mfit/mnit"] = ROOT.TH1D("c_mfit_mnit", "MFIT/MNIT", nbins, 0., -1.)
chistos["mfib/mnib"] = ROOT.TH1D("c_mfib_mnib", "MFIB/MNIB", nbins, 0., -1.)


for fill in fills:
    # Fill cuts
    if fillReport.getFillPeakInstLumi(fill) < minPeakInstLumi:
        #print "Low FillPeakInstLumi for fill", fill
        continue
    if fillReport.getFillField(fill) < minBfield:
        #print "Low magnetic f5ield for fill", fill
        continue
    if fillReport.getFillDuration(fill) < deltaT:
        #print "Short fill", fill
        continue
    
    inputFile = inputPattern.replace('__XXX__', str(fill))
    try:
        fin = ROOT.TFile(inputFile,'READ')
        #print "Processing fill", fill
    except IOError:
        print "Cannot open file", inputFile
        continue
    t = fin.Get("t")      

    
    tsColl = -10000

    for i in range(0, t.GetEntries()):
        nb = t.GetEntry(i)
        if nb < 0:
            continue
        
        #Start of collisions 
        if tsColl < 0:
            if t.rates[refIndx] > rateCut:
                tsColl = t.tstamp  
        if tsColl < 0 or t.tstamp < tsColl + deltaT:
            continue
            

        ROOT.gROOT.cd()
        for key in histos.keys():            
            (nom, denom) = key.split('/')
            
            if t.rates[detInd[denom]] > 0. and t.rates[detInd[nom]] > 0. :
                histos[key].Fill(t.rates[detInd[nom]]/t.rates[detInd[denom]])
                chistos[key].Fill(t.rates[detInd[nom]]*detCf[nom]/t.rates[detInd[denom]]/detCf[denom])
                
print "=================DONE================="

=================DONE=================


##Plot statistical ratios (no calibration factors)

In [46]:
cp = ROOT.TCanvas("cp", "+Z", 800, 800)
cp.Divide(2,2)
cp.cd(1)
histos["pnit/pnib"].Draw()
cp.cd(2)
histos["pfit/pfib"].Draw()
cp.cd(3)
histos["pfit/pnit"].Draw()
cp.cd(4)
histos["pfib/pnib"].Draw()
cp.Update()
#
cm = ROOT.TCanvas("cm", "-Z", 800, 800)
cm.Divide(2,2)
cm.cd(1)
histos["mnit/mnib"].Draw()
cm.cd(2)
histos["mfit/mfib"].Draw()
cm.cd(3)
histos["mfit/mnit"].Draw()
cm.cd(4)
histos["mfib/mnib"].Draw()
cm.Update()

TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: cp
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: cm


##Plot statistical ratios (with calibration factors)

In [59]:
ccp = ROOT.TCanvas("ccp", "+Z", 800, 800)
ccp.Divide(2,2)
ccp.cd(1)
chistos["pnit/pnib"].Draw()
ccp.cd(2)
chistos["pfit/pfib"].Draw()
ccp.cd(3)
chistos["pfit/pnit"].Draw()
ccp.cd(4)
chistos["pfib/pnib"].Draw()
ccp.Update()
#
ccm = ROOT.TCanvas("ccm", "-Z", 800, 800)
ccm.Divide(2,2)
ccm.cd(1)
chistos["mnit/mnib"].Draw()
ccm.cd(2)
chistos["mfit/mfib"].Draw()
ccm.cd(3)
chistos["mfit/mnit"].Draw()
ccm.cd(4)
chistos["mfib/mnib"].Draw()
ccm.Update()

TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: ccp
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: ccm


In [49]:
#Fit 
chistos["pnit/pnib"].Fit("gaus")
print chistos["pnit/pnib"].GetMean(), "+/", chistos["pnit/pnib"].GetMeanError()

print chistos["pnit/pnib"].GetRMS()

print '%.3f' % chistos["pnit/pnib"].GetMean(), '\t', '%.3f' % histos["pnit/pnib"].GetMean()


0.798986821289 +/ 3.3135415475e-05
0.0151832533707
0.799 	0.717


In [60]:
print "Ratio\t\tNo CF\tWith CF" 
for key in sorted(histos.keys()):
    print key.upper(), '\t', '%.3f' % histos[key].GetMean(), '\t', '%.3f' % chistos[key].GetMean()

Ratio		No CF	With CF
MFIB/MNIB 	1.012 	0.717
MFIT/MFIB 	1.435 	1.351
MFIT/MNIT 	1.660 	1.285
MNIT/MNIB 	0.875 	0.753
PFIB/PNIB 	1.200 	1.208
PFIT/PFIB 	0.650 	0.654
PFIT/PNIT 	1.088 	0.990
PNIT/PNIB 	0.717 	0.799
